In [2]:
%load_ext autoreload
%autoreload 2
%pdb
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Automatic pdb calling has been turned OFF


In [3]:
import sys
sys.path.insert(0, '/home/ubuntu/Documents/infembed')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/data')
from data._core.spotcheck import get_spotcheck_dataloader, get_blindspots_df
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/models')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/')
# sys.path.insert(0, )
from infembed.embedder._core.fast_kfac_embedder import FastKFACEmbedder
import torchvision
from torch.utils.data import Subset, DataLoader, default_collate, Dataset
from torchvision.models import ResNet18_Weights, resnet18
import torch.nn as nn
from infembed.clusterer._core.sklearn_clusterer import SklearnClusterer
from infembed.clusterer._core.faiss_clusterer import FAISSClusterer
from infembed.clusterer._core.rule_clusterer import RuleClusterer
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd
import torch
from typing import List
from infembed.utils.common import Data
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable
from torch import Tensor
from typing import Tuple
from models._utils.common import init_linear
from lightning.pytorch.callbacks import EarlyStopping
import lightning.pytorch as pl
import lightning as L
import torch.nn.functional as F
import numpy as np

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


### figure out device to compute embeddings on ###

In [4]:
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print('device:', DEVICE)

device: cuda:0


### define data
We will define the following:
- `eval_dataset`: `Dataset` for evaluation data.  This is used to retrieve individual examples for displaying.  This should be the same data for which the embeddings we load later were computed
- `eval_dataloader`: `DataLoader` for evaluation data.  This is used to compute metadata for the evaluation data

In [5]:
normalize = ResNet18_Weights.IMAGENET1K_V1.transforms()

def collate_fn(examples):
    return tuple([_x.to(device=DEVICE) for _x in default_collate([(normalize(__x[0]), __x[1]) for __x in examples])])

BATCH_SIZE = 32
NUM_EVAL = 50000
eval_dataset = Subset(
    torchvision.datasets.ImageNet("/home/ubuntu/Documents/infembed/files/imagenet", split="val"),
    range(NUM_EVAL),
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=collate_fn, batch_size=BATCH_SIZE)

### define model ###
only used to get metadata

In [6]:
model = resnet18(weights=ResNet18_Weights.DEFAULT).to(device=DEVICE)
# model.load_state_dict(ResNet18_Weights.IMAGENET1K_V1.get_state_dict(progress=True))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### load embeddings for evaluation data ###

In [8]:
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_timing_experiments/0/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_kfac_global_projection_dim/1/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_kfac_last_layer/2/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_gradient_last_layer/0/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/scripts/run_embedder/outputs/imagenet_dim_1000/0/embeddings.pt'
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/examples/imagenet/hydra_outputs/run_embeddings/imagenet_pca_gradient/4/embeddings.pt' # pca gradient 1000 dim
# embeddings_path = '/home/ubuntu/Documents/infembed/infembed/examples/imagenet/hydra_outputs/run_embeddings/imagenet_arnoldi/3/embeddings.pt' # arnoldi 
embeddings_path = '/home/ubuntu/Documents/infembed/examples/imagenet/hydra_outputs/run_embeddings/imagenet_arnoldi_layer4_eval_no_shuffle/arnoldi_dim_100/embeddings.pt'
embeddings = torch.load(open(embeddings_path, 'rb'))
embeddings.shape

torch.Size([50000, 66])

optionally, we can only keep some elements of the embedding (relevant for `ArnoldiEmbedder`)

In [9]:
DIMS_TO_KEEP = 50
if DIMS_TO_KEEP is not None:
    # embeddings = embeddings[:, -DIMS_TO_KEEP:]
    embeddings = embeddings[:, :DIMS_TO_KEEP]

optionally, we can normalize the embeddings to have the same norm

In [10]:
NORMALIZE = True
if NORMALIZE:
    embeddings = embeddings / torch.linalg.norm(embeddings, dim=1)[:, None]

In [11]:
if True:
    # already assume embeddings fit in memory, so no point in using incremental methods
    pass

we then package them into a `Data` instance, which contains all kinds of data that could possibly be used to do the subsequent clustering, i.e. including tabular metadata as well.

In [12]:
data = Data(embeddings=embeddings)

### compute metadata for evaluation data ###
this will be the ingredient needed to display the clusters.  later on, it will also be used by the rule-based clusterer.  therefore, we also add it to the running `Data` instance for easy access.

In [13]:
def _get_predictions_and_labels(_model, dataloader):
    dfs = []
    for batch in tqdm(dataloader):
        prediction_prob = (
            torch.nn.functional.softmax(_model(*batch[:-1]), dim=1)
            .detach()
            .to(device="cpu")
        )
        prediction_label = torch.argmax(prediction_prob, dim=1).to(device="cpu")
        label = batch[-1].to(
            device="cpu"
        )  # assuming batch is a tensor.  if not, can check
        dfs.append(
            pd.DataFrame(
                {
                    "prediction_label": prediction_label,
                    "label": label,
                    "prediction_prob": list(prediction_prob.numpy()),
                }
            )
        )
    df = pd.concat(dfs, axis=0)
    df.index = list(range(len(df)))
    return df

if False:
    metadata = _get_predictions_and_labels(model, eval_dataloader)

read in predictions

In [20]:
import glob
preds_path = "/home/ubuntu/Documents/infembed/examples/imagenet/hydra_outputs/lightning_predict/val/preds"
preds_paths = glob.glob(f"{preds_path}/*")
preds_paths = sorted(preds_paths, key=lambda path: int(path.split('/')[-1].split('.')[0]))
preds = pd.concat([pd.read_csv(path, index_col=0) for path in preds_paths], axis=0)
prediction_label = preds.idxmax(axis=1)
prediction_label

0     817
1     817
2     817
3     817
4     817
     ... 
11    817
12    817
13     92
14    817
15    817
Length: 50000, dtype: object

add in column with human-readable prediction and label names

In [15]:
import json
import pandas as pd

class_index_to_name = pd.read_csv(
    open("/home/ubuntu/Documents/infembed/files/imagenet/imagenet_classes.txt", "r"),
#     sep=" ",
    index_col=None,
    header=None,
    
)
class_index_to_name.columns = ['name']
class_index_to_name.index = list(range(len(class_index_to_name)))
class_index_to_name

,name
0,tench
1,goldfish
2,great white shark
3,tiger shark
4,hammerhead
...,...
995,earthstar
996,hen-of-the-woods
997,bolete
998,ear


change metadata

In [16]:
def rename(index):
    return class_index_to_name.loc[index]['name']

metadata['prediction_label_name'] = metadata['prediction_label'].apply(rename)
metadata['label_name'] = metadata['label'].apply(rename)
metadata

NameError: name 'metadata' is not defined

In [ ]:
data.metadata = metadata
data.dataset = eval_dataset

### define clusterer ###

In [ ]:
if False:
    clusterer = SklearnClusterer(sklearn_clusterer=KMeans(n_clusters=25))
if True:
    clusterer = FAISSClusterer(k=25, spherical=True)

### do the clustering ###

In [ ]:
clusters = clusterer.fit_predict(data)

### define ways to display clusters ###
these will all be functions whose input is a list of list of indices in the evaluation dataset

In [ ]:
from infembed.visualization._core.common import (
    DisplayMetadata,
    DisplayPIL,
    DisplayPredictionAndLabels,
    DisplaySingleExamples,
    PerClusterDisplayer,
    DisplayAccuracy,
)

displayers = [
    PerClusterDisplayer(
        [
            DisplayAccuracy(prediction_col="prediction_label", label_col="label"),
            DisplayPredictionAndLabels(
                prediction_col="prediction_label_name", label_col="label_name"
            ),
            # DisplaySingleExamples(
            #     [
            #         DisplayMetadata(["label_name", "prediction_label_name"]),
            #         DisplayPIL(),
            #     ],
            #     limit=3,
            # ),
        ]
    )
]

### display the clusters ###

In [ ]:
for displayer in displayers:
    displayer(clusters, data)

cluster #0
accuracy: 0.70 (1525/2182)
prediction: {'basset': 8, 'gorilla': 8, 'gibbon': 7, 'Shih-Tzu': 7, 'Polaroid camera': 7, 'jellyfish': 7, 'photocopier': 7, 'abacus': 6, 'macaque': 6, 'white wolf': 6, 'monitor': 6, 'swimming trunks': 6, 'prison': 6, 'jinrikisha': 6, 'altar': 6, 'cowboy hat': 6, 'rule': 6, 'convertible': 6, 'chest': 6, 'boathouse': 6, 'pick': 6, 'china cabinet': 6, 'cauliflower': 6, 'bottlecap': 6, 'bookcase': 6, 'canoe': 6, 'jacamar': 5, 'nail': 5, 'pitcher': 5, 'clumber': 5, 'football helmet': 5, 'nipple': 5, 'crutch': 5, 'English springer': 5, 'catamaran': 5, 'cash machine': 5, 'maillot': 5, 'American alligator': 5, 'bath towel': 5, 'stage': 5, 'handkerchief': 5, 'red wolf': 5, 'Saluki': 5, 'park bench': 5, 'Border collie': 5, 'sea lion': 5, 'mixing bowl': 5, 'hare': 5, 'Gila monster': 5, 'French horn': 5, 'bannister': 5, 'tank': 5, 'alp': 5, 'green snake': 5, 'basenji': 5, 'buckeye': 5, 'magnetic compass': 5, 'puffer': 5, 'loggerhead': 5, 'bee eater': 5, 'chimp

### define rule clusterer ###

In [ ]:
def _accuracy(data):
    return (data.metadata["prediction_label"] == data.metadata["label"]).mean()


def _size(data):
    return len(data)


rule_clusterer = RuleClusterer(
    # clusterer_getter=lambda n_clusters: SklearnClusterer(KMeans(n_clusters=n_clusters)),
    clusterer_getter=lambda n_clusters: FAISSClusterer(k=n_clusters, spherical=True),
    cluster_rule=lambda data: _accuracy(data) < 0.5 and _size(data) >= 10,
    stopping_rule=lambda data: _size(data) < 50,
    max_depth=7,
    branching_factor=3,
)

### do the rule clustering ###

In [ ]:
rule_clusters = rule_clusterer.fit_predict(data)

WARNING clustering 96 points to 3 centroids: please provide at least 117 training points
WARNING clustering 93 points to 3 centroids: please provide at least 117 training points
WARNING clustering 63 points to 3 centroids: please provide at least 117 training points
WARNING clustering 88 points to 3 centroids: please provide at least 117 training points
WARNING clustering 71 points to 3 centroids: please provide at least 117 training points
WARNING clustering 88 points to 3 centroids: please provide at least 117 training points
WARNING clustering 65 points to 3 centroids: please provide at least 117 training points
WARNING clustering 51 points to 3 centroids: please provide at least 117 training points
WARNING clustering 65 points to 3 centroids: please provide at least 117 training points
WARNING clustering 82 points to 3 centroids: please provide at least 117 training points
WARNING clustering 91 points to 3 centroids: please provide at least 117 training points
WARNING clustering 54

### display the rule clusters

In [ ]:
for displayer in displayers:
    displayer(rule_clusters, data)

cluster #0
accuracy: 0.47 (9/19)
prediction: {'goldfish': 1, 'hatchet': 1, 'box turtle': 1, 'loggerhead': 1, 'Gila monster': 1, 'sulphur-crested cockatoo': 1, 'starfish': 1, 'vending machine': 1, 'fireboat': 1, 'garbage truck': 1, 'grand piano': 1, 'pickelhaube': 1, 'padlock': 1, 'plate rack': 1, 'yawl': 1, 'scoreboard': 1, 'military uniform': 1, 'wing': 1, 'bell pepper': 1}
label: {'goldfish': 1, 'bald eagle': 1, 'box turtle': 1, 'African crocodile': 1, 'sidewinder': 1, 'sulphur-crested cockatoo': 1, 'king crab': 1, 'ballpoint': 1, 'fireboat': 1, 'garbage truck': 1, 'grand piano': 1, 'military uniform': 1, 'padlock': 1, 'plate rack': 1, 'schooner': 1, 'scoreboard': 1, 'suit': 1, 'sundial': 1, 'broccoli': 1}
cluster #1
accuracy: 0.45 (5/11)
prediction: {'sturgeon': 1, 'pelican': 1, 'Arabian camel': 1, 'colobus': 1, 'disk brake': 1, 'sax': 1, 'fire screen': 1, 'web site': 1, 'mixing bowl': 1, 'space shuttle': 1, 'digital watch': 1}
label: {'tiger shark': 1, 'oystercatcher': 1, 'Arabian 